# **ANÁLISIS EXPLORATORIO DE DATOS PARTE 2**

1. Tipos de datos 

2. Manejando valores vacíos

3. Renombramiento columnas e índices

4. Combinando dataframes

5. Manejando fechas

<img src="../imgs/foto-dia-03.jpg" width="500px" height="300px">

En esta clase vamos a trabajar con el dataset de canciones en Spotify en 2023, ejecuta la siguiente celda para ver las columnas del dataframe.

In [ ]:
import pandas as pd

spotify = pd.read_csv("datasets/spotify-2023.csv", encoding='latin-1')

spotify.describe()

### **1. Tipos de datos**

El tipo de datos de una columna en un DataFrame o una Serie se conoce como **dtype**.

Puedes utilizar la propiedad dtype para obtener el tipo de una columna específica. Por ejemplo, podemos obtener el dtype de la columna precio en el DataFrame revisiones:

In [ ]:
spotify["artist_count"].dtype

Alternativamente, la propiedad dtypes devuelve el dtype de cada columna del DataFrame:

In [ ]:
spotify.dtypes

Los tipos de datos nos dicen algo sobre cómo pandas está almacenando los datos internamente. float64 significa que está usando un número de coma flotante de 64 bits; int64 significa un entero de tamaño similar, y así sucesivamente.

Una peculiaridad a tener en cuenta (y que se muestra muy claramente aquí) es que las columnas que consisten enteramente en cadenas no tienen su propio tipo; en su lugar se les da el tipo de objeto.

Es posible convertir una columna de un tipo en otro siempre que dicha conversión tenga sentido utilizando la función astype(). Por ejemplo, podemos transformar la columna de `artist_count` de su actual tipo de datos int64 a un tipo de datos float64:

In [ ]:
spotify.artist_count.astype('float64')

Un índice DataFrame o Series también tiene su propio dtype:

In [ ]:
spotify.index.dtype

### **2. Manejando valores vacíos**

Los valores que faltan en los registros reciben el valor NaN, abreviatura de "Not a Number" (no es un número). Por razones técnicas estos valores NaN son siempre del tipo float64.

Pandas proporciona algunos métodos específicos para datos perdidos. Para seleccionar registros NaN se puede utilizar pd.isnull() (o su compañera pd.notnull()). Esto está pensado para ser usado así:

In [ ]:
spotify[pd.isnull(spotify.key)]

Para saber que columnas tienen valores nulos dentro de nuestro dataframe podemos utilizar el siguiente código que nos dice el porcentaje de valores nulos para cada columna.

In [ ]:
porcentaje_nulos_por_columna = (spotify.isnull().sum() / len(spotify)) * 100
print(porcentaje_nulos_por_columna)

Reemplazar valores NaN es una operación común. Pandas proporciona un método realmente práctico para este problema: `fillna()`. 

fillna() proporciona unas cuantas estrategias diferentes para mitigar tales datos. Por ejemplo, podemos simplemente reemplazar cada NaN por un "Desconocido":

In [ ]:
spotify['key'] = spotify.key.fillna("Unknown")

spotify['key'].value_counts()

O podemos rellenar cada valor que falte con el primer valor no nulo que aparezca en algún momento después del registro dado en la base de datos. Esto se conoce como estrategia de relleno.

También podemos tener un valor no nulo que queramos sustituir. Por ejemplo, supongamos que desde que se publicó este conjunto de datos, Bad Bunny ha cambiado su nombre artístico a Good Bunny, pues esto lo podríamos reflejar así.

In [ ]:
spotify["artist(s)_name"] = spotify["artist(s)_name"].replace("Bad Bunny", "Good Bunny")

spotify.loc[spotify["artist(s)_name"] == "Good Bunny"]

### **3. Renombrando columnas e índices**

A menudo los datos nos llegan con nombres de columnas, índices u otras convenciones de nomenclatura con las que no estamos satisfechos. Para eso tenemos las funciones `rename` y `rename_axis`

La primera función rename(), permite cambiar los nombres de índices y/o columnas. Por ejemplo, para cambiar la columna nombre de artistas de nuestro conjunto de datos, haríamos:

In [ ]:
spotify.rename(columns={'artist(s)_name': 'artists_name'})

`rename()` permite renombrar valores de índice o columna especificando un parámetro de palabra clave de índice o columna, respectivamente. Admite varios formatos de entrada, pero normalmente el más conveniente es un diccionario de Python. Un ejemplo que lo utiliza para renombrar algunos elementos del índice es el siguiente:

In [ ]:
spotify.rename(index={0: 'firstEntry', 1: 'secondEntry'})

Probablemente renombrará columnas muy a menudo, pero renombrará valores de índice muy raramente. Para eso, `set_index()` suele ser más conveniente ya que normalmente renombraremos el índice con los valores de alguna columna.

Tanto el índice de fila como el índice de columna pueden tener su propio atributo nombre. El método `rename_axis()` puede utilizarse para cambiar estos nombres. Por ejemplo:

In [ ]:
spotify.rename_axis("canciones", axis='rows').rename_axis("columnas", axis='columns')

### **4. Combinando dataframes**

Al realizar operaciones en un conjunto de datos, a veces necesitaremos combinar diferentes DataFrames y/o Series de formas no sencillas. Pandas tiene tres métodos básicos para hacer esto. En orden de complejidad creciente, son `concat()`, `join()` y `merge()`. La mayor parte de lo que puede hacer merge() también se puede hacer de forma más sencilla con join(), por lo que la omitiremos y nos centraremos en las dos primeras funciones.

El método de combinación más sencillo es concat(). Dada una lista de elementos, esta función los unirá a lo largo de un eje.

Esto es útil cuando tenemos datos en diferentes objetos DataFrame o Series pero que tienen los mismos campos (columnas). Un ejemplo, es el que vamos a ver ahora con el conjunto de datos YouTube Videos, que divide los datos en función del país de origen (por ejemplo, Canadá y USA, en este ejemplo). Si queremos estudiar varios países a la vez, podemos utilizar concat() para agruparlos.

In [ ]:
canadian_youtube = pd.read_csv("./datasets/CAvideos.csv")
usa_youtube = pd.read_csv("./datasets/USvideos.csv")

union_youtube = pd.concat([canadian_youtube, usa_youtube])
print(len(union_youtube))
print(len(canadian_youtube))
print(len(usa_youtube))

El combinador intermedio en términos de complejidad es `join()`. join() permite combinar diferentes objetos DataFrame que tienen un índice en común. Por ejemplo, para obtener los vídeos que fueron tendencia el mismo día en Canadá y en USA, podemos hacer lo siguiente:

In [ ]:
left = canadian_youtube.set_index(['title', 'trending_date'])
print(len(left))

right = usa_youtube.set_index(['title', 'trending_date'])
print(len(right))

left_join = left.join(right, lsuffix='_CAN', rsuffix='_USA')

left_join.loc[left_join.likes_USA.notnull()]


Los parámetros lsuffix y rsuffix son necesarios aquí porque los datos tienen los mismos nombres de columna en los conjuntos de datos británicos y estado unidenses. Si no fuera así (porque, por ejemplo, les hubiéramos cambiado el nombre de antemano) no los necesitaríamos.

### **5. Manejando fechas**

En el análisis de datos, a menudo necesitamos trabajar con datos de fecha y hora. Pandas proporciona numerosas funciones y métodos para manejar eficazmente datos de fecha y hora. A continuación, vamos a ver algunas de las funciones más comunes para el manejo de fechas en Pandas:

La función `pd.to_datetime()` se utiliza para convertir una columna a un tipo de dato de fecha y hora. Esto es útil cuando los datos de fecha y hora están en formato de cadena y necesitas convertirlos a un formato reconocido por Pandas.

In [ ]:
canadian_youtube.head(5)

canadian_youtube['publish_time'] = pd.to_datetime(canadian_youtube['publish_time'])

canadian_youtube['trending_date'] = pd.to_datetime(canadian_youtube['trending_date'], format="%y.%d.%m") #y minúscula es año con 2 dígitos y Y año con 4 dígitos.
canadian_youtube.head(5)


Es importante saber que podemos realizar extracciones de cierta parte de la fecha, como el mes o el año y también podemos realizar operaciones aritméticas para sumar o restar días, meses, etc...

In [ ]:
# Extracción de partes de una fecha.
canadian_youtube['year_publish_time'] = canadian_youtube['publish_time'].dt.year #month, day, minute, hour...

# Añadir días a una fecha
canadian_youtube['trending_date_plus_seven'] = canadian_youtube['trending_date'] + pd.Timedelta(days=7)

# Restar meses a una fecha
canadian_youtube['trending_date_less_month'] = canadian_youtube['trending_date'] - pd.DateOffset(months=1)
canadian_youtube.head(5)

Una vez que sabemos extraer ciertas partes de una fecha se nos abre una posibilidad a realizar estudios de agrupación y filtrado por estas partes de la fecha, en especial, esto es muy útil para utilizar el año.

In [ ]:
# Filtro de registros por año
canadian_youtube_2017 = canadian_youtube.loc[canadian_youtube['publish_time'].dt.year == 2017]

# Agrupación de vídeos subidos en un año concreto
canadian_youtube.groupby(canadian_youtube['publish_time'].dt.year)['video_id'].describe()